![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [42]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

student_id      city  ...  training_hours job_change
0        8949  city_103  ...              36        1.0
1       29725   city_40  ...              47        0.0
2       11561   city_21  ...              83        0.0
3       33241  city_115  ...              52        1.0
4         666  city_162  ...               8        0.0

[5 rows x 14 columns]

In [46]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

import numpy as np


#Added a function that converts datatype
def convert_dt(df, column, datatype):
    return df[column].astype(datatype)

ds_jobs_transformed['city_development_index'] = convert_dt(ds_jobs_transformed, 'city_development_index', 'float16')

ds_jobs_transformed['gender'] =  convert_dt(ds_jobs_transformed, 'gender', 'category')

#Stopped implementing the function here coz im lazy but the other rows are right
ds_jobs_transformed['city'] = convert_dt(ds_jobs_transformed, 'city', 'category')

ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].astype('category')
company_size = ['<10','10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+']
ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].cat.set_categories(new_categories = company_size, ordered = True)

ds_jobs_transformed['company_type'] = ds_jobs_transformed['company_type'].astype('category')

ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].astype('category')
last_new_job = ['never', '1', '2', '3', '4', '>4']
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].cat.set_categories(new_categories = last_new_job, ordered = True)

ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].astype('category')
experience = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].cat.set_categories(new_categories = experience, ordered = True)

ds_jobs_transformed['major_discipline'] = ds_jobs_transformed['major_discipline'].astype('category')

ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].astype('category')
level = ['Primary School','High School', 'Graduate', 'Masters', 'Phd']
ds_jobs_transformed['education_level'] =ds_jobs_transformed['education_level'].cat.set_categories(new_categories = level, ordered =True)


ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype('category')
enrolled_university = ['no_enrollment', 'Part time course', 'Full time course']
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].cat.set_categories(new_categories = enrolled_university, ordered =True)

ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype('int32')
ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype('int32')

ds_jobs_transformed['relevant_experience'] = np.where(ds_jobs_transformed['relevant_experience'].str.contains('Has relevant experience'), True, False)

ds_jobs_transformed['job_change'] = np.where(ds_jobs_transformed['job_change'] == 1 , True, False)

student_10_more = ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']
student_1000_companies = ['1000-4999', '5000-9999', '10000+']

ds_jobs_transformed_filtered = ds_jobs_transformed[ds_jobs_transformed['experience'].isin(student_10_more) & ds_jobs_transformed['company_size'].isin(student_1000_companies)]


ds_jobs_transformed = ds_jobs_transformed_filtered

ds_jobs.info()
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me